In [1]:
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
# Definimos el cliente sobre el que podemos hacer el tracking
MLFLOW_TRACKING_URI = "sqlite:///exp-tracking.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [84]:
experiment = client.get_experiment_by_name('nyc-taxi.experiment')

In [85]:
best_run = client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type= ViewType.ACTIVE_ONLY,
        max_results=5,
        order_by=["metrics.rmse ASC"]
        )

In [88]:
best_run[0].info.run_id

'a2204949054048e7bc3f828854e95226'

In [89]:
f'runs/{best_run[0].info.run_id}'

'runs/a2204949054048e7bc3f828854e95226'

In [ ]:
# Crear experimentos
client.create_experiment(name = "my-cool-experiment", artifact_location='./test_artifacts')

'3'

In [33]:
from mlflow.entities import ViewType
# Buscar runs
runs = client.search_runs(
    experiment_ids=[1],
    filter_string= "metrics.rmse <6.8",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [34]:
for  run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: a2204949054048e7bc3f828854e95226, rmse: 6.1914
run id: 3d88811a1714443fb588c6a55ebc2369, rmse: 6.2284
run id: 4893f1d296cd4c749d23311973aa543c, rmse: 6.2690
run id: 1b01034aa0ac4b69908a088247365fee, rmse: 6.2851
run id: 9e0a059ce6cd48f3acb7a63badcb7e79, rmse: 6.2983


In [35]:
# Información de metricas de cada run
runs[0].data

<RunData: metrics={'rmse': 6.191359697372561}, params={'learning_rate': '0.06650908516703558',
 'max_depth': '16',
 'min_child_weight': '9.240451398492997',
 'objective': 'reg:squarederror',
 'reg_alpha': '0.05025684864048788',
 'reg_lambda': '0.10146607789146089',
 'seed': '123'}, tags={'mlflow.runName': 'fearless-crow-689',
 'mlflow.source.name': '/home/pinnzonandres/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'pinnzonandres',
 'model': 'xgboost'}>

In [36]:
# Informacion de cada run
runs[0].info

<RunInfo: artifact_uri=('/mnt/c/Users/ASUS/OneDrive/Desktop/Proyectos/Courses/MLops '
 'ZoomCamp/02-Experiment-Tracking/mlruns/1/a2204949054048e7bc3f828854e95226/artifacts'), end_time=1747007403944, experiment_id='1', lifecycle_stage='active', run_id='a2204949054048e7bc3f828854e95226', run_name='fearless-crow-689', run_uuid='a2204949054048e7bc3f828854e95226', start_time=1747007336185, status='FINISHED', user_id='pinnzonandres'>

In [37]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
# Registro de Modelos 
model_uri = f"runs/{'a2204949054048e7bc3f828854e95226'}"
mlflow.register_model(model_uri= model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747751642812, current_stage='None', description=None, last_updated_timestamp=1747751642812, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/a2204949054048e7bc3f828854e95226', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [44]:
# Busqueda de modelos registrdos
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1747746586937, description='', last_updated_timestamp=1747746587157, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1747746587157, current_stage='None', description='', last_updated_timestamp=1747746587157, name='NYC-regressor', run_id='94aeffab63b348e7a88aa7a81da2f1a0', run_link='', source=('/mnt/c/Users/ASUS/OneDrive/Desktop/Proyectos/Courses/MLops '
  'ZoomCamp/02-Experiment-Tracking/mlruns/1/94aeffab63b348e7a88aa7a81da2f1a0/artifacts/models'), status='READY', status_message=None, tags={}, user_id=None, version=1>], name='NYC-regressor', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1747747944746, description='NYC Taxi Regressor Duration', last_updated_timestamp=1747751642812, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1747747974475, current_stage='Staging', description='', last_updated_timestamp=1747748043959, name='nyc-taxi-regressor', run_id='e787bf00ec0641498c9c411570b224b9', run_

In [52]:
latest_versions = client.get_latest_versions(name='nyc-taxi-regressor')
for version in latest_versions:
    print(f"Version:{version.version}, Stage: {version.current_stage}")

Version:2, Stage: Staging
Version:3, Stage: None


/tmp/ipykernel_11844/1548586273.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name='nyc-taxi-regressor')


In [ ]:
# Update state of model
client.transition_model_version_stage(
    name = 'nyc-taxi-regressor',
    version = 3,
    stage ="Staging",
    archive_existing_versions= False
)

/tmp/ipykernel_11844/2116195754.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1747751642812, current_stage='Staging', description=None, last_updated_timestamp=1747752054261, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/a2204949054048e7bc3f828854e95226', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [64]:
client.search_runs(
    experiment_ids='nyc-taxi'
)

[]

In [55]:
client.update_model_version(
    name = 'nyc-taxi-regressor',
    version= 3,
    description="Cambio a Staging"
)

<ModelVersion: aliases=[], creation_timestamp=1747751642812, current_stage='Staging', description='Cambio a Staging', last_updated_timestamp=1747752130240, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/a2204949054048e7bc3f828854e95226', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [ ]:
# Definir un alias a la versión del modelo registrado
client.set_registered_model_alias(
    name='nyc-taxi-regressor',
    version = 3,
    alias = "Staged"
)

In [ ]:
# Añadir tags a los modelos registrados
client.set_registered_model_tag(
    name = 'nyc-taxi-regressor',
    key = 'Estado',
    value = 'Produccion'
)

In [ ]:
# Descargar artefactos
client.download_artifacts(run_id="e787bf00ec0641498c9c411570b224b9", path='preprocessor', dst_path='.')

'/mnt/c/Users/ASUS/OneDrive/Desktop/Proyectos/Courses/MLops ZoomCamp/02-Experiment-Tracking/preprocessor'

In [ ]:
test_mode

Signature:
client.download_artifacts(
    run_id: str,
    path: str,
    dst_path: Optional[str] = None,
) -> str
Docstring:
Download an artifact file or directory from a run to a local directory if applicable,
and return a local path for it.

Args:
    run_id: The run to download artifacts from.
    path: Relative source path to the desired artifact.
    dst_path: Absolute path of the local filesystem destination directory to which to
        download the specified artifacts. This directory must already exist.
        If unspecified, the artifacts will either be downloaded to a new
        uniquely-named directory on the local filesystem or will be returned
        directly in the case of the LocalArtifactRepository.

Returns:
    Local path of desired artifact.

.. code-block:: python
    :caption: Example

    import os
    import mlflow
    from mlflow import MlflowClient

    features = "rooms, zipcode, median_price, school_rating, transport"
    with open("features.txt", "w") as